In [1]:
import pandas as pd
import numpy as np

## Reading in the Datasets

#### - Concatenating all course data into one large dataset
#### - Merging by the ppsk (unique student ID) to get demographics info

In [2]:
courses_2012 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2012.txt", sep="\t", low_memory = False)
courses_2013 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2013.txt", sep="\t", low_memory = False)
courses_2014 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2014.txt", sep="\t", low_memory=False)
courses_2015 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2015.txt", sep="\t", low_memory = False)
courses_2016 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2016.txt", sep="\t", low_memory = False)
courses_2017 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2017.txt", sep="\t", low_memory = False)
ppsk = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.stem.pathways.2007.2018.txt", sep="\t", low_memory=False)

In [3]:
#concatenating the course data
course_data = pd.concat([courses_2012, courses_2013, courses_2014, courses_2015, courses_2016, courses_2017])
course_data

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,course.title,instructor.name,grade.subtype,grade.type,grade.name,grade.points,student.credit.hours
0,2012 Spring,46441,Graduate,Business,Evening & Weekend MBA,217,Primary,11,9668,104458,Topics In Eap,Felix Vardy,A,Letter Grade,A+,4.0,1.0
1,2012 Spring,67711,Undergraduate,Education,Education,140AC,Primary,1,23725,23725,Liter:Ind & Soc Dev,Glynda Hull,A,Letter Grade,A+,4.0,3.0
2,2012 Spring,114711,Graduate,Classics,Greek,105,Primary,1,15218,15218,New Testament,Trevor Murphy,A,Letter Grade,A+,4.0,4.0
3,2012 Spring,125970,Undergraduate,History,History,125B,Primary,1,39546,39546,Afri-Am 1861-1980,Waldo Martin,A,Letter Grade,A+,4.0,4.0
4,2012 Spring,133388,Graduate,Public Health,Public Health,267D,Primary,1,76316,76316,Health Impact Asses,Edmund Seto; Jason Corburn,A,Letter Grade,A+,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573019,2017 Fall,3142831,Graduate,Mechanical Engineering,Mechanical Engineering,C231A,Secondary,101,39876,259,Exp Adv Ctrl Des I,Edward Zhu,Unknown,Unknown,Unknown,NaN,0.0
573020,2017 Fall,3142833,Graduate,Psychology,Psychology,167AC,Secondary,102,46470,46470,Stigma,Angela Johnson,Unknown,Unknown,Unknown,NaN,0.0
573021,2017 Fall,3143145,Undergraduate,Linguistics,Linguistics,1A,Secondary,101,16608,16608,American Sign Lang,Sherry Hicks,Unknown,Unknown,Unknown,NaN,0.0
573022,2017 Fall,3143267,Graduate,Ethnic Studies,Asian American Studies,128AC,Secondary,103,12944,12944,Muslims In America,Subhasree Banerjee,Unknown,Unknown,Unknown,NaN,0.0


In [4]:
#merging by ppsk
student_data = pd.merge(course_data, ppsk, on = 'ppsk', how='left')
student_data

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,...,degree.division.3,degree.department.3,degree.major.3,degree.college.4,degree.division.4,degree.department.4,degree.major.4,college,division,ppsk.1
0,2012 Spring,46441,Graduate,Business,Evening & Weekend MBA,217,Primary,11,9668,104458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012 Spring,67711,Undergraduate,Education,Education,140AC,Primary,1,23725,23725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012 Spring,114711,Graduate,Classics,Greek,105,Primary,1,15218,15218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012 Spring,125970,Undergraduate,History,History,125B,Primary,1,39546,39546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012 Spring,133388,Graduate,Public Health,Public Health,267D,Primary,1,76316,76316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072169,2017 Fall,3142831,Graduate,Mechanical Engineering,Mechanical Engineering,C231A,Secondary,101,39876,259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3072170,2017 Fall,3142833,Graduate,Psychology,Psychology,167AC,Secondary,102,46470,46470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3072171,2017 Fall,3143145,Undergraduate,Linguistics,Linguistics,1A,Secondary,101,16608,16608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3072172,2017 Fall,3143267,Graduate,Ethnic Studies,Asian American Studies,128AC,Secondary,103,12944,12944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filtering the DataFrame

### - Filter by lower division CS --> (61A, 61B, 70)
### - Make dummy variables
### - Group by ppsk
### - Get average gpa of lower divs

In [22]:
#lower div
student_data = student_data[student_data['undergrad.status']=='Undergraduate']
cs_lower = student_data[student_data['course.number'].isin(['61A', '61B', '70'])]

#turn final.majors into separate tables and turn into one column
fmm1 = pd.DataFrame(data = {'ppsk': cs_lower['ppsk'],'major': cs_lower['final.major.major.1']})
fmm2 = pd.DataFrame(data = {'ppsk': cs_lower['ppsk'],'major': cs_lower['final.major.major.2']})
fmm3 = pd.DataFrame(data = {'ppsk': cs_lower['ppsk'],'major': cs_lower['final.major.major.3']})
fmm4 = pd.DataFrame(data = {'ppsk': cs_lower['ppsk'],'major': cs_lower['final.major.major.4']})

fmm_1 = pd.DataFrame(data = {'ppsk': cs_lower['ppsk'],'major': cs_lower['first.major.major.1']})
fmm_2 = pd.DataFrame(data = {'ppsk': cs_lower['ppsk'],'major': cs_lower['first.major.major.2']})
fmm_3 = pd.DataFrame(data = {'ppsk': cs_lower['ppsk'],'major': cs_lower['first.major.major.3']})
fmm_4 = pd.DataFrame(data = {'ppsk': cs_lower['ppsk'],'major': cs_lower['first.major.major.4']})

declared_majors = fmm_1.combine_first(fmm_2).combine_first(fmm_3).combine_first(fmm_4).combine_first(fmm2).combine_first(fmm3).combine_first(fmm4)


#add column to cs_lower
cs_lower = pd.merge(cs_lower, declared_majors, on = 'ppsk', how='left')


#create dummies
major_dummies = pd.get_dummies(cs_lower[['major']])
# modified_cs = pd.get_dummies(cs_lower[['ppsk', 'instructor.name', 'year.name', 'course.number', 'gender', 'ethnic.l1']]).drop_duplicates() #flexible
modified_cs = pd.get_dummies(cs_lower[['ppsk','course.number', 'gender', 'ethnic.l1']]).drop_duplicates() #flexible


modified_cs



,ppsk,course.number_61A,course.number_61B,course.number_70,gender_Decline to State,gender_Female,gender_Male,ethnic.l1_Asian/Pacific Islander,ethnic.l1_International,ethnic.l1_Underrepresented Minority,ethnic.l1_White/Other
0,847130,0,1,0,0,0,1,1,0,0,0
3,916674,0,1,0,0,0,1,0,0,0,1
6,920733,0,1,0,0,0,1,1,0,0,0
9,937794,0,1,0,0,0,1,0,0,0,1
14,948499,1,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
397161,3124335,1,0,0,0,0,0,0,0,0,0
397167,3125025,0,0,1,0,0,0,0,0,0,0
397169,3127538,1,0,0,0,0,0,0,0,0,0
397172,3131520,0,1,0,0,0,0,0,0,0,0


In [ ]:
declared_majors

In [23]:
#group by ppsk

grouped_courses = modified_cs.groupby('ppsk').agg(max)
# add year, major declaration

new_df = pd.DataFrame(data = { 'ppsk': cs_lower['ppsk'], 'declared': major_dummies['major_L&S Computer Science']}).drop_duplicates()
grouped_courses = grouped_courses.merge(new_df, on = 'ppsk', how = 'left')


#add gpa to modified_cs
gpa = pd.DataFrame(data = cs_lower.groupby(['ppsk'])['grade.points'].mean())

final_table = grouped_courses.merge(gpa, on='ppsk', how = 'left')

In [27]:
len(final_table[final_table['declared'] == 1])
# sum(final_table['declared'])

2902

In [25]:
final_table

,ppsk,course.number_61A,course.number_61B,course.number_70,gender_Decline to State,gender_Female,gender_Male,ethnic.l1_Asian/Pacific Islander,ethnic.l1_International,ethnic.l1_Underrepresented Minority,ethnic.l1_White/Other,declared,grade.points
0,7780,1,0,0,0,0,0,0,0,0,0,0,1.15
1,136248,1,0,0,0,0,0,0,0,0,0,0,NaN
2,294778,0,0,1,0,0,0,0,0,0,0,0,2.70
3,360766,1,0,0,0,0,0,0,0,0,0,0,0.00
4,434810,0,0,1,0,0,0,0,0,0,0,0,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15209,3134805,1,0,0,0,0,1,0,0,0,1,0,2.70
15210,3135896,1,0,0,0,0,0,0,0,0,0,0,2.30
15211,3136455,1,0,0,0,0,0,0,0,0,0,0,4.00
15212,3137103,1,0,0,0,0,0,0,0,0,0,0,4.00


## Making Different Classifiers

### Decision Tree
### Logistic Regression
### Neural Networks
### XBoost

### Add Cross Validation
### Ensemble Learning

In [28]:
import itertools
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [32]:
final_table = final_table.dropna()

X = final_table.drop(columns = ['ppsk', 'declared'])
Y = final_table['declared']

clf = DecisionTreeClassifier(max_depth=3)
clf = clf.fit(X, Y)
y_pred = clf.predict(X)

In [33]:
accuracy_score(Y, y_pred)

0.782324182023317